##Projeto de um controlador Fuzzy PID##

Projete um controlador PID Fuzzy para a seguinte planta:

\begin{equation}
  \frac{dy}{dt} = \frac{1}{\tau_p}\left [ k_p u(t) - y(t)  \right ]
\end{equation}

com $\tau_p $ sendo o último dígito da matrícula do aluno A (por exemplo) e $k_p$ os dois últimos dígitos da matrícula do aluno B (se algum for nulo, considere a unidade). 

Considere o período de amostragem como sendo $0,000u_1$, sendo $u_1$ o último dígito da matrícula do aluno A ou B (se for nulo, considere a unidade).

Com as seguintes regras de inferências:

* R1: Se (erro = EP e d_erro = DEP) então ($k_c$ = PP,  $\tau_i$ = IP, $\tau_d$ = DP)

* R2: Se (erro = EP e d_erro = DEM) então ($k_c$ = PP,  $\tau_i$ = IP, $\tau_d$ = DP)

* R3: Se (erro = EP e d_erro = DEG) então ($k_c$ = PP,  $\tau_i$ = IP, $\tau_d$ = DP)

* R4: Se (erro = EM e d_erro = DEP) então ($k_c$ = PM,  $\tau_i$ = IM, $\tau_d$ = DM)

* R5: Se (erro = EM e d_erro = DEM) então ($k_c$ = PM,  $\tau_i$ = IM, $\tau_d$ = DM)

* R6: Se (erro = EM e d_erro = DEG) então ($k_c$ = PM,  $\tau_i$ = IM, $\tau_d$ = DM)

* R7: Se (erro = EG e d_erro = DEP) então ($k_c$ = PG,  $\tau_i$ = IG, $\tau_d$ = DG)

* R8: Se (erro = EG e d_erro = DEM) então ($k_c$ = PG,  $\tau_i$ = IG, $\tau_d$ = DG)

* R9: Se (erro = EG e d_erro = DEG) então ($k_c$ = PG,  $\tau_i$ = IG, $\tau_d$ = DG)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [ ]:
# Modelo da planta a ser controlada
n = 100 # número de pontos a serem simulados
tf = 50.0 # tempo final
SP_start = 1.0 # quando inicia o set point
def planta(y,t,u):
    Kp = 4.0      # penúltimo dígito da matrícula do aluno A ou B (se for nulo considere a unidade)
    tau_p = 3.0   # último dígito da matrícula do aluno A ou B (se for nulo considere a unidade)
    theta_p = 1.0  
    if t < (theta_p + SP_start):
       dydt = 0.0 # atraso 
    else:
       dydt = (1.0/tau_p) * ((Kp*u) - y)
    return dydt

In [ ]:
# Modelo discretizado (método de Euler) da planta a ser controlada 

def planta_discreta(y, u, Ts, N):
    # parâmetros do modelo
    Kp = 4.0
    tau_p = 3.0
    theta_p = 1.0

    y[0] = 0.0

    # Simulação
    for n in range (N+1):
        y[n + 1] = y[n] + (Ts/tau_p)*(4*u[n] - y[n])
    
    return y
 
Ts = 0.1
Tstop = 50
N = int(Tstop/Ts) 
y = np.zeros(N + 2)
u = np.ones(N + 2)

t = np.arange (0, Tstop + 2*Ts, Ts ) #Cria a série temporal
y = planta_discreta(y, u, Ts, N)

plt.scatter(t, y)
plt.title('Simulação do modelo discreto')
plt.xlabel('n')
plt.ylabel('y[n]')
plt.grid()
plt.axis([ 0 , Tstop , 0 , max(y) + .5 ])
plt.show()

In [ ]:
def pid(Kc,tauI,tauD):
    t = np.linspace(0,tf,n) # vetor tempo
    P = np.zeros(n) # incializa o termo proporcional
    I = np.zeros(n) # inicializa o termo integrativo
    D = np.zeros(n) # inicializa o termo derivativo
    e = np.zeros(n) # inicializa o erro
    U = np.zeros(n) # inicializa a saída do controlador
    PV = np.zeros(n) # inicializa a variável do processo
    SP = np.zeros(n) # inicializa o set point (referência)
    SP_step = int(SP_start/(tf/(n-1))+1) 
    SP[0:SP_step] = 0.0 # define a referência inicial
    SP[SP_step:n] = 4.0 # referência final
    y0 = 0.0 # condição inicial

    for i in range(1,n):
        # simulação do processo
        ts = [t[i - 1], t[i]] # intervalo de tempo
        y = odeint(planta,y0,ts,args = (U[i - 1],)) # resolve numericamente a EDO
        y0 = y[1] # armazena a nova condição inicial
        # calcula o sinal de controle do PID a ser aplicado a planta
        PV[i] = y[1] # armazena a variável do processo
        e[i] = SP[i] - PV[i] # calcula o erro = SP - PV
        dt = t[i] - t[i-1] # calcula o delta t
        P[i] = Kc * e[i] # calcula o termo proporcional
        I[i] = I[i-1] + (Kc/tauI) * e[i] * dt # calcula o termo integrativo
        D[i] = -Kc * tauD * (PV[i]-PV[i-1])/dt # calcula o termo derivativo
        U[i] = P[i] + I[i] + D[i] # Sinal de controle 
        
    return t, e, SP, PV, U

kc = 0.5
tau_i = 1
tau_d = 0.0
[t, erro, r, y, u] = pid(kc,tau_i,tau_d)
# plot PID response
plt.figure(1,figsize=(15,7))
plt.subplot(2,2,1)
plt.plot(t,r,'k-',linewidth=2,label='Set point (SP)')
plt.plot(t,y,'g--',linewidth=2,label='Variável do processo (PV)')
plt.grid()
plt.legend(loc='best')
plt.subplot(2,2,2)
plt.plot(t,erro,'r:',linewidth=2,label='Erro (e = SP - PV)')
plt.grid()
plt.legend(loc='best')
plt.subplot(2,2,3)
plt.plot(t,u,'b:',linewidth=2,label='Sinal de controle u(t)')
plt.grid()
plt.legend(loc='best')
plt.xlabel('tempo')  
plt.show()

In [ ]:
# Implementação do controlador fuzzy: Funções de pertinência, fuzzificação e defuzzificação

def trimf(x, param):
    
def trapmf(x, param):
    
def gaussmf(x, param):
    
def gbellmf(x, param):
    
def fuzz(x0, y, mA_list, mB_list):
    A0 = np.zeros(len(mA_list))
    Bi = np.zeros([y.size, len(mA_list)])
    out = np.zeros(y.size)
    for i in range(len(mA_list)):
        fA = mA_list[i][0]
        param_A = mA_list[i][1]
        A0[i] = fA(float(x0), param_A)
        fB = mB_list[i][0]
        param_B = mB_list[i][1]
        for j in range(y.size):
            Bi[j,i] = min(A0[i], fB(float(y[j]), param_B))
            out[j] = max(Bi[j,:])
    return(out)

def defuzz(y, mf, option):
    if option == 'centroid':
        num = 0
        den = 0
        for i in range(y.size):
            num = num + y[i]*mf[i]
            den = den + mf[i]
        y0 = num/den
        return y0
    
    elif option == 'bisector':
        area = 0
        aux = 0
        for i in range(y.size - 1):
            area = area + (y[i+1] - y[i]) * ((mf[i+1] + mf[i])/2)
        for i in range(y.size):
            aux = aux + (y[i+1] - y[i]) * ((mf[i+1] + mf[i])/2)
            if (aux >= area/2):
                return y[i]
        
    elif option == 'MOM':
        mf_max = max(mf)
        acum = 0
        n = 0
        for i in range(y.size):
            if (mf[i] == mf_max):
                acum = acum + y[i]
                n = n + 1
        return acum/n
              
    else:
        return -1
    

In [ ]:
# Definindo as variáveis linguísticas de entrada e saída

# Universo de discurso
erro = 
d_erro = 

Kc = 
tauI = 
tauD = 

# Funções de pertinência para o erro


# Funções de pertinência para a variação do erro (Delta_erro)


# Funções de pertinência para os parâmetros do controlador Out_Kp, Out_tauI, Out_tauD


In [ ]:
# Gráfico das funções de pertinência para o erro e variação do erro
